Install Dependencies

In [ ]:
%pip install gymnasium[accept-rom-license]

In [ ]:
%pip install gymnasium[atari]

In [ ]:
%pip install stable-baselines3[extra]

In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Import Dependencies

In [ ]:
import os
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3 import A2C


Constants

In [ ]:
ENV_NAME = 'ALE/Adventure-v5'
ENV_N_STACKS = 9
ENV_SEED = 0
ENV_RENDER_MODE = 'human'

CALLBACK_LOG_DIR = '\logs\\adventure'
CALLBACK_CHECKPOINT_DIR = '\models\\adventure'
CALLBACK_CHECK_FREQ = 10000

VERBOSE = 1

ALGORITHM_BEST_MODEL_NAME = f'{ENV_N_STACKS}_stacks_adventure_v5_best_model.zip'
ALGORITHM_POLICY = 'CnnPolicy'
ALGORITHM_DEVICE = 'cuda'
ALGORITHM_TOTAL_TIMESTEPS = 1000000

ALGORITHM_RENDER_MODEL = False
ALGORITHM_RENDER_EPISODES = 10
ALGORITHM_NEW_MODEL = True
ALGORITHM_LOAD_MODEL = False

MESSAGE_RENDERING_MODEL = 'RENDERING MODEL'
MESSAGE_LOADING_MODEL = 'LOADING MODEL'
MESSAGE_TRAINING_NEW_MODEL = 'NEW MODEL'


Creating the Env

In [ ]:
vec_env = make_atari_env(ENV_NAME, n_envs=ENV_N_STACKS, seed=ENV_SEED)
vec_env = VecFrameStack(vec_env, n_stack=ENV_N_STACKS)

Creating Callback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=VERBOSE):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
    
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, '{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

callback = TrainAndLoggingCallback(check_freq=CALLBACK_CHECK_FREQ, save_path=CALLBACK_CHECKPOINT_DIR)

Loading, Training and Rendering

In [ ]:
model = None

if ALGORITHM_RENDER_MODEL is True:
    print(MESSAGE_RENDERING_MODEL)
    model = A2C.load(ALGORITHM_BEST_MODEL_NAME, env=vec_env)

    for ep in range(ALGORITHM_RENDER_EPISODES):
        obs = vec_env.reset()
        for ep in range(ALGORITHM_RENDER_EPISODES):
            action, _ = model.predict(obs, deterministic=False)
            obs, rewards, dones, info = vec_env.step(action)
            vec_env.render(ENV_RENDER_MODE)
    vec_env.close()

elif ALGORITHM_LOAD_MODEL is True:
    print(MESSAGE_LOADING_MODEL)
    model = A2C.load(ALGORITHM_BEST_MODEL_NAME, env=vec_env)
    model.learn(total_timesteps=ALGORITHM_TOTAL_TIMESTEPS, callback=callback) 

elif ALGORITHM_NEW_MODEL is True:
    print(MESSAGE_TRAINING_NEW_MODEL)
    model = A2C(ALGORITHM_POLICY, vec_env, tensorboard_log = CALLBACK_LOG_DIR, verbose=VERBOSE, device=ALGORITHM_DEVICE)
    model.learn(total_timesteps=ALGORITHM_TOTAL_TIMESTEPS, callback=callback) 